In [65]:
import pandas as pd
import numpy as np
from datetime import date
import time
import gcld3
import warnings 
import emoji
import re
import nltk
from nltk.corpus import stopwords
from scripts.scripts import *
warnings.filterwarnings("ignore") 

pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', 200)

In [2]:
df1 = pd.read_json('/Volumes/ML_projects/Phase4data/Iphone12.json', lines=True)
df2 = pd.read_csv('/Volumes/ML_projects/Phase4data/apple_silicon_SEARCH.csv')
df3 = pd.read_csv('/Volumes/ML_projects/Phase4data/M1_MacBook_SEARCH.csv')
df4 = pd.read_csv('/Volumes/ML_projects/Phase4data/new_macbook_SEARCH.csv')
df5 = pd.read_csv('/Volumes/ML_projects/Phase4data/Iphone_12_Pro_SEARCH.csv', error_bad_lines=False, engine="python")
df6 = pd.read_csv('/Volumes/ML_projects/Phase4data/new_Apple_SEARCH.csv', error_bad_lines=False, engine="python")
df7 = pd.read_csv('/Volumes/ML_projects/Phase4data/new_iphone_SEARCH.csv', error_bad_lines=False, engine="python")
df8 = pd.read_csv('/Volumes/ML_projects/Phase4data/Silicon_SEARCH.csv', error_bad_lines=False, engine="python")

Skipping line 129970: ',' expected after '"'
Skipping line 235431: ',' expected after '"'
Skipping line 269778: ',' expected after '"'
Skipping line 276452: ',' expected after '"'
Skipping line 201421: ',' expected after '"'
Skipping line 74247: ',' expected after '"'
Skipping line 197383: ',' expected after '"'
Skipping line 222434: ',' expected after '"'
Skipping line 234201: ',' expected after '"'
Skipping line 256441: ',' expected after '"'
Skipping line 137005: ',' expected after '"'
Skipping line 157999: ',' expected after '"'
Skipping line 158000: ',' expected after '"'
Skipping line 185665: ',' expected after '"'


Add search terms to each dataframe before concatinating. 

In [3]:
df1['search_term'] = 'iphone 12'
df2['search_term'] = 'apple silicon'
df3['search_term'] = 'M1 macbook'
df4['search_term'] = 'new macbook'
df5['search_term'] = 'iphone 12 pro'
df6['search_term'] = 'new apple'
df7['search_term'] = 'new iphone'
df8['search_term'] = 'silicon'

In [4]:
frames = [df1, df2, df3, df4, df5, df6, df7, df8]

In [5]:
main = pd.concat(frames, ignore_index=True)

In [6]:
main.shape

(2425258, 37)

# Remove Foreign Language Tweets

In [7]:
main = main.loc[main['language'] == 'en']

In [8]:
main.shape

(1582752, 37)

# Convert Emojis to text

In [9]:
# new clean_tweet columns
main['c_tweet'] = 'cleaned'

In [11]:
main['c_tweet'] = main.tweet.apply(emoji.demojize)

# Remove numbers

In [50]:
main['c_tweet'] = main.c_tweet.apply(cleaner_text)

# Remove special Characters

In [51]:
main['c_tweet'] = main.c_tweet.apply(clean_hash)

In [36]:
main.to_csv('/Volumes/ML_projects/Phase4data/appletweet.csv')

##### Drop Duplicate Tweets

In [52]:
# If tweet is repeated multiple times only keep last occurance.
main = main.drop_duplicates(subset='c_tweet', keep='last')
main.shape

(985657, 38)

# Create Stopwords

In [55]:
stop_words=list(set(stopwords.words("english")))

In [56]:
stop_words.extend(["ll", "please retweet", "help rt", "visit us",
                   "please subscribe", "for sale", "enter here", 
                   "to win", "from iphone", "huge giveaway", "new arrival", 
                   "win an", "unlocked now", "new available", "giveaway", 
                   "subscribe", "retweet", "easter egg"])

In [57]:
eda_stopwords = ["iphone", "iphone pro", "pro max", "youtube", "youtuber", "youtubers", 
                 "also", "asap", "hmmm","hello", "haha", "dont", "imac", "ipad", 
                 "ipadair", "ipads", "iphonecase", "iphonemini", "iphonepro", "iphonepromax", 
                 "iphones", "iphoneth", "ipod",  "lmao", "lmfao", "macbook", "macbookair",
                 "macbookpro", "macbooks", "macosbigsur", "macmini", "magsafe", 
                 "okay", "promax", "wahala", "wont", 'yall','yeah', 'apple', 'skin', 'tone', "man", "pro", "face", "loudly",]

stop_words.extend(eda_stopwords)

In [58]:
len(stop_words)

242

# Tokenize/Lemmatize

In [67]:
lemmatizer = nltk.stem.WordNetLemmatizer()
tokenizer = nltk.tokenize.TweetTokenizer()

In [68]:
def lemmatize_text(text):
     return [lemmatizer.lemmatize(word.replace(" ", "")) for word in tokenizer.tokenize(text)]

In [69]:
main['tokenized'] = main.c_tweet.apply(lemmatize_text)

In [ ]:
#Remove stop words, and words with only 2 chars. 
main['filtered'] = main['tokenized'].apply(lambda x: [item for item in x if item not in stop_words and len(item)>2])

# Use VADER sentiment framework to add tags

In [ ]:
main['sentiment'] = influence.c_tweet.apply(sentiment_analyzer_scores)